In [1]:
import pandas as pd

df = pd.read_csv("stats19CycleCollisions2022.csv", header=0)
df = df.rename(columns={
    'longitude.x': 'longitude',
    'latitude.x': 'latitude',
    'date.x': 'date'
})

In [3]:
# determining number of vehicles involved in each collision - assume this will be 2 the majority of the time
accident_summary = df.groupby("accident_index").agg({"number_of_vehicles": "max",
                                                     "vehicle_reference": pd.Series.nunique,
                                                     "casualty_reference": pd.Series.nunique})

accident_summary.head()

,number_of_vehicles,vehicle_reference,casualty_reference
accident_index,,,
2022010352601,1,1,1
2022010352612,2,2,1
2022010352651,2,2,1
2022010352661,2,2,1
2022010352702,2,2,1


In [4]:
# cases where number_of_vehicles not equal to vehicle count
accident_summary[accident_summary['number_of_vehicles'] != accident_summary['vehicle_reference']]
# -> no cases

,number_of_vehicles,vehicle_reference,casualty_reference
accident_index,,,


In [5]:
accident_summary[accident_summary['casualty_reference'] > 1]
# -> 222 cases
# examples include:
#  - 1 bike and no other vehicle, 2 casualties 
#  - bike and motorbike, 2 casualties
#  - bike, motorbike and car, 2 casualties from bike and motorbike
#  - 2 bikes, 2 casualties
#  - 2 bikes and van, 2 casualties
# First, for collisions where there is more than 1 bike casualty, I will create a separate collision for each bike casualty
# Then for collisions where there is more than 1 vehicle, I will take only one vehicle based on the following hierarchy:
#    1. HGV
#    2. Agricultural vehicle
#    3. Bus
#    4. Van
#    5. Car
#    6. Motorbike

,number_of_vehicles,vehicle_reference,casualty_reference
accident_index,,,
2022010353008,2,2,2
2022010353026,3,3,2
2022010353698,2,2,2
2022010353786,3,3,2
2022010354052,3,3,2
...,...,...,...
2022461244748,2,2,2
2022481232155,2,2,2
2022481262319,1,1,2


In [6]:
# creating vehicle subtype hierarchy
df['vehicle_subtype'] = [
    "4. Van" if vehicle_type[0] in ["VAN / GOODS 3.5 TONNES MGW OR UNDER", "MINIBUS (8 - 16 PASSENGER SEATS)"]
    else "1. HGV" if "GOODS" in vehicle_type[0]
    else "2. Agricultural vehicle" if vehicle_type[0] == "AGRICULTURAL VEHICLE"
    else "3. Bus" if vehicle_type[0] == "BUS OR COACH (17 OR MORE PASS SEATS)"
    else "5. Car" if "CAR" in vehicle_type[0]
    else "6. Motorbike" if "MOTORCYCLE" in vehicle_type[0]
    else "7. Bike" if vehicle_type[0] == "PEDAL CYCLE"
    else "8. Unknown"
    for vehicle_type in zip(df['vehicle_type'].str.upper())
]

In [9]:
# creating a separate collision for each bike casualty
casualty_vars = [
    'longitude', 
    'latitude', 
    'date',
    'day_of_week',
    'time', 
    'first_road_class',
    'road_type', 
    'speed_limit',
    'junction_detail', 
    'junction_control', 
    'second_road_class',
    'pedestrian_crossing_human_control', 
    'pedestrian_crossing_physical_facilities', 
    'light_conditions',
    'weather_conditions', 
    'road_surface_conditions',
    'special_conditions_at_site', 
    'carriageway_hazards',
    'urban_or_rural_area',
    'RH2M', 
    'T2M', 
    'PRECTOTCORR',
    'WS2M',
    'sex_of_casualty', 
    'age_of_casualty', 
    'age_band_of_casualty',
    'casualty_severity',
    'casualty_home_area_type', 
    'casualty_imd_decile', 
    'lsoa_of_casualty'
]
vehicle_vars = [
    'vehicle_type', 
    'vehicle_subtype',
    'towing_and_articulation',
    'vehicle_manoeuvre',
    'vehicle_location_restricted_lane', 
    'junction_location',
    'skidding_and_overturning', 
    'hit_object_in_carriageway',
    'vehicle_leaving_carriageway', 
    'hit_object_off_carriageway',
    'first_point_of_impact', 
    'vehicle_left_hand_drive',
    'engine_capacity_cc', 
    'propulsion_code',
    'age_of_vehicle'
]
driver_vars = [
    'journey_purpose_of_driver', 
    'sex_of_driver', 
    'age_of_driver',
    'age_band_of_driver',
    'driver_imd_decile',
    'driver_home_area_type', 
    'lsoa_of_driver'
]
unique_casualties = df[(df['vehicle_type'] == 'Pedal cycle') & ~pd.isnull(df['casualty_reference'])][['accident_index', 'casualty_reference'] + casualty_vars].drop_duplicates().reset_index(drop=True)
unique_casualties['accident_index_2'] = unique_casualties.index
unique_casualties.head()

,accident_index,casualty_reference,longitude,latitude,date,day_of_week,time,first_road_class,road_type,speed_limit,...,PRECTOTCORR,WS2M,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty,accident_index_2
0,2022010352601,1.0,-0.088278,51.534038,2022-01-01,Saturday,02:10,A,Dual carriageway,20,...,0.39,4.76,Male,41.0,36 - 45,Slight,Data missing or out of range,Data missing or out of range,-1,0
1,2022010352612,1.0,-0.147037,51.586480,2022-01-01,Saturday,10:45,B,Roundabout,20,...,0.39,4.76,Male,64.0,56 - 65,Slight,Urban area,More deprived 40-50%,E01000888,1
2,2022010352651,1.0,-0.059434,51.527071,2022-01-02,Sunday,12:47,A,Single carriageway,30,...,3.68,5.61,Male,23.0,21 - 25,Slight,Urban area,Most deprived 10%,E01004316,2
3,2022010352661,1.0,-0.228878,51.534284,2022-01-02,Sunday,18:30,Unclassified,Single carriageway,30,...,3.68,5.61,Male,59.0,56 - 65,Serious,Urban area,More deprived 30-40%,E01000534,3
4,2022010352702,1.0,-0.086519,51.631613,2022-01-03,Monday,10:20,Unclassified,Single carriageway,30,...,0.19,4.40,Male,71.0,66 - 75,Slight,Data missing or out of range,Data missing or out of range,-1,4


In [11]:
df_expanded = pd.merge(unique_casualties, df[['accident_index', 'casualty_reference', 'number_of_vehicles'] + vehicle_vars + driver_vars], how="inner", on="accident_index", suffixes=["", "_y"])
df_expanded = df_expanded[((df_expanded['casualty_reference'] != df_expanded['casualty_reference_y']) & (df_expanded['number_of_vehicles'] > 1)) | (df_expanded['number_of_vehicles'] == 1)]\
    .drop(columns=['casualty_reference', 'casualty_reference_y', 'accident_index', 'number_of_vehicles'])
df_expanded.head()

,longitude,latitude,date,day_of_week,time,first_road_class,road_type,speed_limit,junction_detail,junction_control,...,engine_capacity_cc,propulsion_code,age_of_vehicle,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,-0.088278,51.534038,2022-01-01,Saturday,02:10,A,Dual carriageway,20,Not at junction or within 20 metres,Data missing or out of range,...,Data missing or out of range,Undefined,-1,Not known,Male,41.0,36 - 45,Data missing or out of range,Data missing or out of range,-1
1,-0.147037,51.586480,2022-01-01,Saturday,10:45,B,Roundabout,20,Roundabout,Give way or uncontrolled,...,1197,Petrol,6,Not known,Male,52.0,46 - 55,More deprived 40-50%,Urban area,E01001962
3,-0.059434,51.527071,2022-01-02,Sunday,12:47,A,Single carriageway,30,Other junction,Give way or uncontrolled,...,1395,Petrol,6,Not known,Male,27.0,26 - 35,More deprived 20-30%,Urban area,E01001749
5,-0.228878,51.534284,2022-01-02,Sunday,18:30,Unclassified,Single carriageway,30,Not at junction or within 20 metres,Data missing or out of range,...,1991,Petrol,4,Not known,Female,49.0,46 - 55,Less deprived 30-40%,Urban area,E01000493
7,-0.086519,51.631613,2022-01-03,Monday,10:20,Unclassified,Single carriageway,30,T or staggered junction,Give way or uncontrolled,...,1796,Petrol,18,Not known,Female,41.0,36 - 45,Most deprived 10%,Urban area,E01001554


In [12]:
# keep 1 vehicle per collision, based on hierarchy
df_expanded = df_expanded.sort_values(['accident_index_2', 'vehicle_subtype'])
df_expanded = df_expanded.groupby('accident_index_2').nth(0)
df_expanded.head()

,longitude,latitude,date,day_of_week,time,first_road_class,road_type,speed_limit,junction_detail,junction_control,...,engine_capacity_cc,propulsion_code,age_of_vehicle,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,-0.088278,51.534038,2022-01-01,Saturday,02:10,A,Dual carriageway,20,Not at junction or within 20 metres,Data missing or out of range,...,Data missing or out of range,Undefined,-1,Not known,Male,41.0,36 - 45,Data missing or out of range,Data missing or out of range,-1
1,-0.147037,51.586480,2022-01-01,Saturday,10:45,B,Roundabout,20,Roundabout,Give way or uncontrolled,...,1197,Petrol,6,Not known,Male,52.0,46 - 55,More deprived 40-50%,Urban area,E01001962
3,-0.059434,51.527071,2022-01-02,Sunday,12:47,A,Single carriageway,30,Other junction,Give way or uncontrolled,...,1395,Petrol,6,Not known,Male,27.0,26 - 35,More deprived 20-30%,Urban area,E01001749
5,-0.228878,51.534284,2022-01-02,Sunday,18:30,Unclassified,Single carriageway,30,Not at junction or within 20 metres,Data missing or out of range,...,1991,Petrol,4,Not known,Female,49.0,46 - 55,Less deprived 30-40%,Urban area,E01000493
7,-0.086519,51.631613,2022-01-03,Monday,10:20,Unclassified,Single carriageway,30,T or staggered junction,Give way or uncontrolled,...,1796,Petrol,18,Not known,Female,41.0,36 - 45,Most deprived 10%,Urban area,E01001554


### creating new features

In [ ]:
# creating date and time features
df_expanded['time_period'] = (df_expanded.time.str.slice(start=0, stop=2).astype('int') // 4)
df_expanded['time_period'] = (df_expanded['time_period'] * 4).astype('str') + ':00 - ' + ((df_expanded['time_period'] + 1) * 4).astype('str') + ':00'

df_expanded['date'] = pd.to_datetime(df_expanded.date)
df_expanded['month'] = df_expanded['date'].dt.month
df_expanded['season'] = [
    'spring' if month in [3, 4, 5]
    else 'summer' if month in [6, 7, 8]
    else 'autumn' if month in [9, 10, 11]
    else 'winter'
    for month
    in df_expanded['month']
]

df_expanded = df_expanded.drop(columns=['date', 'month'])